### ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png)

## Project Capstone

### Reducing Electricity Costs in Romania: Predictive and Reinforcement Learning for Solar Power Optimization

---

[README](../README.md) | **Part 1: Cleaning** | [Part 2: EDA](02_EDA.ipynb) | [Part 3: Solar Energy Prediction Models](03_Model_Solar_Energy.ipynb) | [Part 4: Electricity Cost Prediction Models](04_Model_Electricity_Cost.ipynb) | [Part 5: Electricity Cost Prediction Using Time Series Models](05_Model_Electricity_Cost_ts.ipynb)

---

### Introduction
- The data was extracted from 3 sources:
  - Weather in Bucharest, Romania: **[Visual Crossing's weather data services](https://www.visualcrossing.com/weather/weather-data-services/Bucharest,Romania/metric/)**
  - Electricity Cost per Hour: **[Thingler](https://thingler.io/country/Romania)**, by interpreting the bar charts and making personal notes
  - Electricity Production per Hour: **[Kaggle](https://www.kaggle.com/datasets/stefancomanita/hourly-electricity-consumption-and-production)**
- All three datasets cover the period from January 2024 to March 2024 on hourly basis. The data for March 31, 2024 was intentionally removed due to missing one hour because of daylight saving adjustments.
- Finally, all three datasets were cleaned and merged into a single file, [electricity_romania.csv]('../data/cleaned/electricity_romania.csv'). The data dictionary is provided below.

### Data Dictionary
| Column Name          | Description                                                      | Data Type       |
|----------------------|------------------------------------------------------------------|-----------------|
| `datetime`           | Date and time of the record                                      | `datetime64[ns]`|
| `temp`               | Temperature in Bucharest (Celsius)                               | `float64`       |
| `feelslike`          | Feels-like temperature (Celsius)                                 | `float64`       |
| `dew`                | Dew point temperature (Celsius)                                  | `float64`       |
| `humidity`           | Relative humidity percentage                                     | `float64`       |
| `windgust`           | Wind gust speed                                                  | `float64`       |
| `windspeed`          | Wind speed                                                       | `float64`       |
| `winddir`            | Wind direction                                                   | `float64`       |
| `sealevelpressure`   | Sea level pressure                                               | `float64`       |
| `cloudcover`         | Percentage of cloud cover                                        | `float64`       |
| `visibility`         | Visibility distance (km)                                         | `float64`       |
| `has_snow`           | Indicates if there is snow (binary: 0 = no, 1 = yes)             | `int32`         |
| `has_solarradiation` | Indicates if there is solar radiation (binary: 0 = no, 1 = yes)  | `int32`         |
| `icon_cloudy`        | Icon representing cloudy weather (binary: 0 = no, 1 = yes)       | `float64`       |
| `icon_fog`           | Icon representing fog weather (binary: 0 = no, 1 = yes)          | `float64`       |
| `icon_partly_cloudy` | Icon representing partly cloudy weather (binary: 0 = no, 1 = yes)| `float64`       |
| `icon_rain`          | Icon representing rain weather (binary: 0 = no, 1 = yes)         | `float64`       |
| `icon_snow`          | Icon representing snow weather (binary: 0 = no, 1 = yes)         | `float64`       |
| `ckwh`               | Electricity cost (cent of euro) per hour in Romania              | `float64`       |
| `consumption`        | Electricity consumption per hour                                 | `int64`         |
| `production`         | Electricity production per hour                                  | `int64`         |
| `nuclear`            | Nuclear energy production (MWs) per hour                         | `int64`         |
| `wind`               | Wind energy production (MWs) per hour                            | `int64`         |
| `hydroelectric`      | Hydroelectric energy production (MWs) per hour                   | `int64`         |
| `oil_gas`            | Oil and gas energy production (MWs) per hour                     | `int64`         |
| `coal`               | Coal energy production (MWs) per hour                            | `int64`         |
| `solar`              | Solar energy production (MWs) per hour                           | `int64`         |
| `biomass`            | Biomass energy production (MWs) per hour                         | `int64`         |            | `int64`         |


### Import & Cleaning

#### Essential Libraries

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

#### Weather Data:
- The data was extracted from **[Visual Crossing's weather data services](https://www.visualcrossing.com/weather/weather-data-services/Bucharest,Romania/metric/)**, covering a period of three months.
- The extracted data is saved into three files: [bucharest_weather_jan24.csv](../data/raw/bucharest_weather_jan24.csv), [bucharest_weather_feb24.csv](../data/raw/bucharest_weather_feb24.csv), and [bucharest_weather_mar24.csv](../data/raw/bucharest_weather_mar24.csv).
- Some columns that were meaningless, contained too many zero values, or had too many `NaN`.
- `snowdepth` and `solarradiation` were converted to binary.
- The `icon` column was converted with `OneHotEncoder`.
- The cleaned data is saved into [weather_clean.csv]('../data/cleaned/weather_clean.csv').

In [2]:
# Import weather data from three files as DataFrames
weather_jan = pd.read_csv('../data/raw/bucharest_weather_jan24.csv')
weather_feb = pd.read_csv('../data/raw/bucharest_weather_feb24.csv')
weather_mar = pd.read_csv('../data/raw/bucharest_weather_mar24.csv')

In [3]:
# Check data shapes
weather_jan.shape, weather_feb.shape, weather_mar.shape

((744, 24), (696, 24), (743, 24))

In [4]:
# Concatenate those DataFrames
weather = pd.concat([weather_jan, weather_feb, weather_mar]
                    , axis = 0
                    , ignore_index = True
                   )

In [5]:
weather.head(1)

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,...,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,"Bucharest,Romania",2024-01-01T00:00:00,3.4,3.4,2.9,96.47,0.0,0,NaN,0.0,...,1018.1,0.0,4.4,0.0,0.0,0.0,NaN,Clear,clear-night,"F0727,15422099999,15419099999,15455099999,1542..."


In [6]:
# Check the number of zeros, NaNs, number of unique values
percent_zeros = (weather == 0).mean() * 100
percent_null = weather.isnull().mean() * 100
unique_values = weather.nunique()

# Create a DataFrame with data type, % zeros, % missing values, and number of unique values
column_info = pd.DataFrame({
    'Data Type': weather.dtypes
    , '% Zeros': percent_zeros
    , '% Missing Values': percent_null
    , 'Unique Values': unique_values
})

column_info

,Data Type,% Zeros,% Missing Values,Unique Values
name,object,0.000000,0.000000,2
datetime,object,0.000000,0.000000,2183
temp,float64,0.229043,0.000000,300
feelslike,float64,0.320660,0.000000,318
dew,float64,0.549702,0.000000,218
humidity,float64,0.000000,0.000000,1816
precip,float64,96.930829,0.000000,57
precipprob,int64,96.930829,0.000000,2
preciptype,object,0.000000,96.472744,3
snow,float64,99.450298,0.000000,10


In [7]:
# Drop columns with too many zeros, missing values, too few unique values, or meaningless data

weather.drop(columns = ['name'                 # meaningless data: Bucharest,Romania; bucharest 
                        , 'precip'             # too many zeros
                        , 'precipprob'         # too many zeros
                        , 'preciptype'         # too many missing values
                        , 'snow'               # too many zeros
                        , 'solarenergy'        # too many zeros
                        , 'uvindex'            # Only zeros and NaNs
                        , 'severerisk'         # All Zeros
                        , 'conditions'         # meaningless
                        , 'stations'           # meaningless
                       ]
             , errors = 'ignore'
             , inplace = True)

In [8]:
# Convert to binary and drop the original columns
weather['has_snow'] = (weather['snowdepth'] > 0).astype(int)
weather['has_solarradiation'] = (weather['solarradiation'] > 0).astype(int)

weather.drop(columns = ['snowdepth'
                        , 'solarradiation'
                       ]
             , errors = 'ignore'
             , inplace = True)

In [9]:
# Map 'partly-cloudy-day' and 'partly-cloudy-night' to 'partly-cloudy'
# Map 'clear-night' and 'clear-day' to 'clear'

weather['icon'] = weather['icon'].replace({
    'partly-cloudy-night': 'partly_cloudy',
    'partly-cloudy-day': 'partly_cloudy',
    'clear-night': 'clear',
    'clear-day': 'clear'
})

# Check the value counts
weather['icon'].value_counts(dropna = False)

icon
cloudy           876
partly_cloudy    791
clear            437
rain              52
snow              15
fog               12
Name: count, dtype: int64

In [10]:
# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Fit and transform the 'icon' column
encoded_icons = encoder.fit_transform(weather[['icon']])

# Create a DataFrame with the encoded columns
encoded_df = pd.DataFrame(encoded_icons, columns = encoder.get_feature_names_out(['icon']))

# Concatenate the original weather DataFrame with the encoded columns, and drop the original 'icon' and 'icon_clear' columns
weather = pd.concat([weather, encoded_df], axis=1).drop(['icon', 'icon_clear'], axis=1)

# Check the result
weather.head()

,datetime,temp,feelslike,dew,humidity,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,has_snow,has_solarradiation,icon_cloudy,icon_fog,icon_partly_cloudy,icon_rain,icon_snow
0,2024-01-01T00:00:00,3.4,3.4,2.9,96.47,6.1,3.7,19.0,1018.1,0.0,4.4,0,0,0.0,0.0,0.0,0.0,0.0
1,2024-01-01T01:00:00,3.1,3.1,2.6,96.54,6.5,3.7,18.0,1017.7,0.0,1.6,0,0,0.0,0.0,0.0,0.0,0.0
2,2024-01-01T02:00:00,2.1,2.1,1.8,97.83,6.1,3.4,10.0,1017.5,0.0,1.6,0,0,0.0,0.0,0.0,0.0,0.0
3,2024-01-01T03:00:00,1.6,1.6,1.4,98.49,6.5,0.3,354.0,1017.2,0.0,1.6,0,0,0.0,0.0,0.0,0.0,0.0
4,2024-01-01T04:00:00,1.5,1.5,1.2,97.81,6.8,0.3,315.0,1016.9,0.0,1.6,0,0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Visibility has 1 NaNs
# weather.isnull().sum()[weather.isnull().sum() > 0]

# Output:
# visibility    1
# dtype: int64

# Impute with median
weather['visibility'] = weather['visibility'].fillna(weather['visibility'].median())

In [12]:
# Formatting datetime
weather['datetime'] = pd.to_datetime(weather['datetime'])

In [13]:
# Filter the DataFrame to include only 01 January 2024 to 30 March 2024
weather = weather[(weather['datetime'] > '2023-12-31 23:59:59') & (weather['datetime'] < '2024-03-31 00:00:00')]

# Save the DataFrame to a CSV file
weather.to_csv('../data/cleaned/weather_clean.csv', index = False)

#### Electricity Cost per Hour
- The data was manually extracted from **[Thingler](https://thingler.io/country/Romania)** by interpreting the bar charts and making personal notes and saved into [hourly_data.csv]('../data/raw/hourly_data.csv').
- The data has 2,184 rows and 2 columns (`dt` and `c_kwH`) and no missing values.
- The data requires converting `dt` to datetime format and renaming columns for easier use.
- The cleaned data is saved into [electricity_cost_clean.csv]('../data/cleaned/electricity_cost_clean.csv').

In [14]:
# Import Electricity Cost per hour data as DataFrame
ecph = pd.read_csv('../data/raw/hourly_data.csv')

In [15]:
# ecph.shape  
# Output:
# (2184, 2)

# ecph.columns
# Output:
# Index(['dt', 'c_kwH'], dtype='object')

# ecph.dtypes
# Output:
# dt        object
# c_kwH    float64
# dtype: object

# ecph.isnull().sum()
# Output:
# dt       0
# c_kwH    0
# dtype: int64

In [16]:
# Original data
# 	dt	c_kwH
# 0	01/01/2024 0:00	1.07

# Convert dt from object to datetime
ecph['dt'] = pd.to_datetime(ecph['dt'], format='%d/%m/%Y %H:%M')

In [17]:
# Filter the DataFrame to include only 01 January 2024 to 30 March 2024
ecph = ecph[(ecph['dt'] > '2023-12-31 23:59:59') & (ecph['dt'] < '2024-03-31 00:00:00')]

# Renaming Columns
ecph = ecph.rename(columns = {'dt': 'datetime', 'c_kwH': 'ckwh'})

# save the DataFrame to a CSV file
ecph.to_csv('../data/cleaned/electricity_cost_clean.csv', index = False)

#### Electricity Production per Hour
- The data was extracted from **[Kaggle](https://www.kaggle.com/datasets/stefancomanita/hourly-electricity-consumption-and-production)** and saved into [electricityConsumptionAndProduction.csv](../data/raw/electricityConsumptionAndProduction.csv).  
- The dataset covers the period from January 2019 to March 2024, has no missing values, and requires formatting `DateTime` as a datetime format.  
- The data is filtered to include only January 2024 to March 2024, and the columns are renamed for easier use
- The cleaned data is saved into [electricity_production_clean.csv]('../datacleaned//electricity_production_clean.csv'). 


In [18]:
# Import Electricity Production per hour data as DataFrame
epph = pd.read_csv('../data/raw/electricityConsumptionAndProduction.csv')

In [19]:
epph.dtypes

DateTime         object
Consumption       int64
Production        int64
Nuclear           int64
Wind              int64
Hydroelectric     int64
Oil and Gas       int64
Coal              int64
Solar             int64
Biomass           int64
dtype: object

In [20]:
epph.isnull().sum()

DateTime         0
Consumption      0
Production       0
Nuclear          0
Wind             0
Hydroelectric    0
Oil and Gas      0
Coal             0
Solar            0
Biomass          0
dtype: int64

In [21]:
# Formatting column DateTime to datetime datatype
epph['DateTime'] = pd.to_datetime(epph['DateTime'])

In [22]:
# Filter the DataFrame to include only 01 January 2024 to 30 March 2024
epph = epph[(epph['DateTime'] > '2023-12-31 23:59:59') & (epph['DateTime'] < '2024-03-31 00:00:00')]

# Renaming Columns and save the DataFrame to a CSV file
epph = epph.rename(columns = {'DateTime': 'datetime'
                              , 'Consumption': 'consumption'
                              , 'Production': 'production'
                              , 'Nuclear': 'nuclear'
                              , 'Wind': 'wind'
                              , 'Hydroelectric': 'hydroelectric'
                              , 'Oil and Gas': 'oil_gas'
                              , 'Coal': 'coal'
                              , 'Solar': 'solar'
                              , 'Biomass': 'biomass'
                             }).sort_values(by = 'datetime').reset_index(drop = True)

# Save the DataFrame to a CSV file
epph.to_csv('../data/cleaned/electricity_production_clean.csv', index = False)

#### Merging All 3 DataFrames
- The three DataFrames can be merged on the `datetime` column, which is common across all of them.
- The merged data is saved into [electricity_romania.csv]('../data/cleaned/electricity_romania.csv').

In [23]:
weather.shape, ecph.shape, epph.shape

((2160, 18), (2160, 2), (2160, 10))

In [24]:
# Merge weather and ecph on 'datetime'
merged_df = pd.merge(weather, ecph, on = 'datetime', how = 'outer')

# Merge the result with epph on 'datetime'
merged_df = pd.merge(merged_df, epph, on = 'datetime', how = 'outer')

In [25]:
# Save the DataFrame to a CSV file
merged_df.to_csv('../data/cleaned/electricity_romania.csv', index = False)

In [26]:
merged_df.dtypes

datetime              datetime64[ns]
temp                         float64
feelslike                    float64
dew                          float64
humidity                     float64
windgust                     float64
windspeed                    float64
winddir                      float64
sealevelpressure             float64
cloudcover                   float64
visibility                   float64
has_snow                       int32
has_solarradiation             int32
icon_cloudy                  float64
icon_fog                     float64
icon_partly_cloudy           float64
icon_rain                    float64
icon_snow                    float64
ckwh                         float64
consumption                    int64
production                     int64
nuclear                        int64
wind                           int64
hydroelectric                  int64
oil_gas                        int64
coal                           int64
solar                          int64
b